In [2]:
!pip install pandas

    100% |████████████████████████████████| 10.1MB 1.8MB/s eta 0:00:01
    100% |████████████████████████████████| 17.3MB 1.0MB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 25.0MB/s ta 0:00:01


In [3]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.6.13

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          85 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge


geopy-1.19.0         | 53 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ########################

In [23]:
df = pd.read_csv("https://raw.githubusercontent.com/ninjananjo/Clothing-Store-Competition/master/StoreLocations.csv")
df.shape

(16, 56)

In [24]:
df = df[['TOWN','NAME','ID','COUNTRY','Address1','Postcode','Host']]
df = df[df['ID'].str.startswith("TS")]

In [33]:
#Add lontitude and lattitude data
addresses = df['Postcode']
addresses_alt = df['Address1']
latitude = []
longitude = []

for address in addresses:
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address+", UK")
    location_alt = geolocator.geocode(addresses_alt+", UK")
    try:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    
    except AttributeError:
        
        try:
            latitude.append(location_alt.latitude)
            longitude.append(location_alt.longitude)
        except:    
            latitude.append(0)
            longitude.append(0)

        
df['Latitude']= latitude
df['Longitude']= longitude

In [44]:
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df

5


,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host,Latitude,Longitude
10,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
11,LONDON (STRAND),STRAND,TS0032,ENG,60/64 The Strand,WC2N 5LR,TS0032,51.509692,-0.122906
12,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873,51.514098,-0.094914
13,LONDON (WHITE CITY),WHITE CITY,TS0874,ENG,2039-2041,W12 7GE,TS0874,51.507417,-0.221361
14,LONDON (KNIGHTSBRIDGE),KNIGHTSBRIDGE,TS1054,ENG,70 Brompton Road,SW3 1ER,TS1054,51.499774,-0.164215


In [45]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 200 # define radius
categoryId = '4bf58dd8d48988d103951735'

In [51]:

#What are the most popular places nearby?

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    
    region_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        #url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name and average price 
        for i in range(0,resultsVen.shape[0]):
            region_list.append(name)
                    
        region_list = pd.DataFrame(region_list, columns=['Store ID'])
        
        nearbyVenues = pd.concat([region_list, resultsVenCat], axis=1)

    return(nearbyVenues)

In [53]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for region in df['ID'].unique():
    region_venues = df[df['ID']==region]
    regions_venues = regions_venues.append((getNearbyVenues(names=region_venues['ID'],
                                                  latitudes=region_venues['Latitude'],
                                                  longitudes=region_venues['Longitude'],
                                                  radius=radius)),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

In [57]:
pd.set_option('display.max_rows', 500)
regions_venues

,Region Name,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,TS0001,4b7979bff964a5205ffb2ee3,Topshop,12,"[214-216 Oxford St, London, Greater London, W1...",51.515441,-0.141262,Women's Store
1,TS0001,4bd52dcf637ba593881bf670,Topman,23,"[214-216 Oxford St (36-38 Great Castle St), Lo...",51.515602,-0.141031,Men's Store
2,TS0001,4ac518f0f964a520afad20e3,NikeTown,33,"[236 Oxford St, London, Greater London, W1C 1D...",51.515488,-0.141824,Sporting Goods Shop
3,TS0001,4b796f06f964a5204ef92ee3,Miss Selfridge,22,"[36 Great Castle St, Oxford Circus, Greater Lo...",51.515547,-0.141030,Women's Store
4,TS0001,4bd970a1e914a593467556fa,Gap,42,"[223-225 Oxford St, London, Greater London, W1...",51.515510,-0.140741,Clothing Store
5,TS0001,4b7979eaf964a5206ffb2ee3,Mango,50,"[225 Oxford St, Marylebone, Greater London, W1...",51.515473,-0.140628,Clothing Store
6,TS0001,4ad45d7af964a520a5e720e3,Urban Outfitters,50,"[200 Oxford St, London, Greater London, W1D 1N...",51.515567,-0.140621,Clothing Store
7,TS0001,4c3335b47cc0c9b63961f19a,Fresh Collection,13,"[237 Oxford Street, City of Westminster, Great...",51.515655,-0.141372,Shoe Store
8,TS0001,4ad718f7f964a520ac0821e3,H&M,69,"[271 Regent Street, London, Greater London, W1...",51.515478,-0.142351,Clothing Store
9,TS0001,4bead0c4ca97d13adb926c77,Tezenis,47,"[266/270 Regent St (Oxford St), Soho, Greater ...",51.515230,-0.141833,Lingerie Store
